# Make mass catalogs using spectroscopic redshifts

In [1]:
import joblib
from pathlib import Path


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.table import Table, vstack, join


#Define cosmology
from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)

### Cat Paths

In [2]:
release = "everest"

In [3]:
# file_name = f"zall-tilecumulative-{release}.fits"
# file_name = f"zall-pix-{release}.fits"
file_name = "zpix-sv3-dark.fits"

In [4]:
cat_path = Path(f"/global/cfs/cdirs/desi/spectro/redux/{release}/zcatalog")
out_path = Path(f"/global/cscratch1/sd/bid13/{release}_stellar_masses")

In [5]:
def fluxToMag(flux):
    return 22.5 - 2.5 * np.log10(flux)

def mw_transmission(cat,band):
    band_coeffs = {"u":3.995, "g": 3.214, "r": 2.165, "i": 1.592, "z": 1.211, "Y": 1.064,
              "W1": 0.184,"W2": 0.113,"W3": 0.0241,"W4": 0.00910}
    A = band_coeffs[band]*cat["EBV"]
    return 10**(A/-2.5)

read in cat and get extinction corrected mags

In [6]:
cat = Table.read(cat_path/file_name)

cat["MW_TRANSMISSION_G"] = mw_transmission(cat,"g")
cat["MW_TRANSMISSION_R"] =  mw_transmission(cat,"r")
cat["MW_TRANSMISSION_Z"] = mw_transmission(cat,"z")
cat["MW_TRANSMISSION_W1"] = mw_transmission(cat,"W1")
cat["MW_TRANSMISSION_W2"] = mw_transmission(cat,"W2")

cat["logmass"] = -99.0
cat["gmag"] = fluxToMag(cat["FLUX_G"] / cat["MW_TRANSMISSION_G"])
cat["rmag"] = fluxToMag(cat["FLUX_R"] / cat["MW_TRANSMISSION_R"])
cat["zmag"] = fluxToMag(cat["FLUX_Z"] / cat["MW_TRANSMISSION_Z"])
cat["w1mag"] = fluxToMag(cat["FLUX_W1"] / cat["MW_TRANSMISSION_W1"])
cat["w2mag"] = fluxToMag(cat["FLUX_W2"] / cat["MW_TRANSMISSION_W2"])

<ipython-input-5-d8de523ab563>:2: RuntimeWarning: divide by zero encountered in log10
  return 22.5 - 2.5 * np.log10(flux)
<ipython-input-5-d8de523ab563>:2: RuntimeWarning: invalid value encountered in log10
  return 22.5 - 2.5 * np.log10(flux)


Load Pre-trained Random Forest models

In [7]:
#MODELS TRAINED USING MGC REDSHIFT
regrf_n = joblib.load(f'/global/cfs/cdirs/desi/users/rongpu/ls_dr9.0_photoz/stellar_mass/rf/rf_20210206_north-mgc_redshift.pkl')
regrf_s = joblib.load(f'/global/cfs/cdirs/desi/users/rongpu/ls_dr9.0_photoz/stellar_mass/rf/rf_20210206_south-mgc_redshift.pkl')
#MODELS TRAINED USING LEGACY SURVEY PHOTO-z
# regrf_n = joblib.load(f'/global/cfs/cdirs/desi/users/rongpu/ls_dr9.0_photoz/stellar_mass/rf/rf_20210206_north.pkl')
# regrf_s = joblib.load(f'/global/cfs/cdirs/desi/users/rongpu/ls_dr9.0_photoz/stellar_mass/rf/rf_20210206_south.pkl')

/global/homes/b/bid13/.local/lib/python3.8/site-packages/sklearn/base.py:324: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 0.23.1 when using version 1.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/global/homes/b/bid13/.local/lib/python3.8/site-packages/sklearn/base.py:324: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 0.23.1 when using version 1.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Mass prediction only possible for objects with all the fluxes

In [8]:
good_mask = (np.isfinite(cat["gmag"]) & np.isfinite(cat["rmag"]) & np.isfinite(cat["zmag"]))
good_mask &= (np.isfinite(cat["w1mag"]) & np.isfinite(cat["w2mag"]))
# good_mask &= (cat["FIBERSTATUS"]==0)
mask_north = (cat["PHOTSYS"]==b"N")
mask_south = (cat["PHOTSYS"]==b"S")

In [9]:
#North
mask = good_mask & mask_north
data = np.column_stack((cat['gmag']-cat['rmag'], cat['rmag']-cat['zmag'], cat['zmag']-cat['w1mag'], cat['w1mag']-cat['w2mag'], cat['Z']))
data = data[mask]
distmod = cosmo.distmod(cat['Z'])
distmod = distmod[mask]
m_to_l_predict = regrf_n.predict(data)
cat["logmass"][mask] = m_to_l_predict + 0.4*(4.45-(cat['zmag'][mask] - distmod.to_value() ))

<ipython-input-9-3495d2f7ddb3>:3: RuntimeWarning: invalid value encountered in subtract
  data = np.column_stack((cat['gmag']-cat['rmag'], cat['rmag']-cat['zmag'], cat['zmag']-cat['w1mag'], cat['w1mag']-cat['w2mag'], cat['Z']))


In [10]:
#South
mask = good_mask & mask_south
data = np.column_stack((cat['gmag']-cat['rmag'], cat['rmag']-cat['zmag'], cat['zmag']-cat['w1mag'], cat['w1mag']-cat['w2mag'], cat['Z']))
data = data[mask]
distmod = cosmo.distmod(cat['Z'])
distmod = distmod[mask]
m_to_l_predict = regrf_s.predict(data)
cat["logmass"][mask] = m_to_l_predict + 0.4*(4.45-(cat['zmag'][mask] - distmod.to_value() ))

<ipython-input-10-f8fd00987317>:3: RuntimeWarning: invalid value encountered in subtract
  data = np.column_stack((cat['gmag']-cat['rmag'], cat['rmag']-cat['zmag'], cat['zmag']-cat['w1mag'], cat['w1mag']-cat['w2mag'], cat['Z']))


In [11]:
cat.write(out_path / ("stellar_mass_specz_" + file_name))